In [1]:
import pandas as pd

In [6]:
html_string = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050.'

In [7]:
dfs = pd.read_html(html_string)

In [9]:
df = dfs[0]

In [10]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Remove rows where Borough Not Assigned

In [11]:
df = df[df.Borough != 'Not assigned']

In [12]:
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


## Group rows with the same Borough and all Neighbourhoods in one row

In [13]:
df = df.groupby('Postcode')['Neighbourhood'].agg({'Borough':'first','Neighbourhood': ', '.join}).reset_index()

/Users/ellu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  """Entry point for launching an IPython kernel.


In [14]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Rouge,"Rouge, Malvern"
1,M1C,Highland Creek,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Guildwood,"Guildwood, Morningside, West Hill"
3,M1G,Woburn,Woburn
4,M1H,Cedarbrae,Cedarbrae


## Assign Neighbourhood to Borough where it's Not Assigned

In [15]:
df[df.Neighbourhood == 'Not assigned'].Neighbourhood = df[df.Neighbourhood == 'Not assigned'].Borough

In [16]:
df.shape

(103, 3)

## Rename columns

In [28]:
df.columns = ['PostalCode', 'Borough', 'Neighbourhood']

In [29]:
df.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Rouge,"Rouge, Malvern"
1,M1C,Highland Creek,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Guildwood,"Guildwood, Morningside, West Hill"
3,M1G,Woburn,Woburn
4,M1H,Cedarbrae,Cedarbrae


## Get Coordinates 

In [30]:
df2 = pd.read_csv('Geospatial_Coordinates.csv')

In [31]:
df2.columns = ['PostalCode', 'Lattitude', 'Longitude']

In [32]:
df2.head()

,PostalCode,Lattitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merge Neighbourhood dataframe with coordinates dataframe

In [33]:
res = pd.merge(df, df2)

In [38]:
res.head()

,PostalCode,Borough,Neighbourhood,Lattitude,Longitude
0,M1B,Rouge,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Highland Creek,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Guildwood,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Woburn,Woburn,43.770992,-79.216917
4,M1H,Cedarbrae,Cedarbrae,43.773136,-79.239476


## Install folium for creating map

In [39]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Folium installed and imported!')

Solving environment: done

## Package Plan ##

  environment location: /Users/ellu/opt/anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.8.3                |   py37hc8dfbb8_1         3.0 MB  conda-forge
    python_abi-3.7             |          1_cp37m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following NEW packages will be INSTALLED:

  python_abi         conda-forge/osx-64::python_abi-3.7-1_cp37m

The following packages will be UPDATED:

  conda                                        4.8.2-py37_0 --> 4.8.3-py37hc8dfbb8_1



python_abi-3.7       | 4 KB      | ##################################### | 100% 
conda-4.8.3          | 3.0 MB    | ##################################### | 100% 
Preparing transaction:

## Create Toronto map

In [83]:
map = folium.Map(location=[43.39, -79.20],
zoom_start=11,
width='80%',
height='80%')

In [84]:
folium.TileLayer('Mapbox Bright').add_to(map)

## Use Neighbouthood dataframe for coordinates and popup marks

In [87]:
latitudes = list(res.Lattitude)
longitudes = list(res.Longitude)
labels = list(res.Borough)

for lat, lng, label in zip(latitudes, longitudes, labels):
    folium.Marker([lat, lng], popup=folium.Popup(label, parse_html=True)).add_to(map)    
    

In [82]:
map